The plan:

* Search for "Cherokee"
* Get 100k tweets
* Searches for individual terms:
    * 'Pocahontas'
    * 'warren'
    * 'grandmother'
    * 'grandma'
    * 'reservation'
    * 'indian'
    * 'blood'
    * 'not offended'
    * 'cheekbones'
    * 'squaw'
    * 'princess'
    * 'sexy'
    * 'hottie'

In [1]:
import math
import datetime
import time
import searchtweets
import numpy as np
import pandas as pd

import string
import warnings
import datetime
warnings.filterwarnings('ignore')
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import seaborn as sns
from IPython.display import display
from wordcloud import WordCloud, STOPWORDS

[nltk_data] Downloading package stopwords to /home/keras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_tweet_object(search_term, start_date, end_date, num_of_tweets, include_retweets=True):
    
    '''
    takes:
    
    * date range
    * screen name
    * number of tweets to return (500 per page)
       
    input search term as a string
    
    input search range as datetime.datetime(20xx, 1, 20)
    
    include_retweets defaults is True. to exclude retweets set to False.
    
    returns: 
    
    * tweets object with replies to input screen name 
   
    '''
    
    
    search_args = searchtweets.load_credentials('twitter_keyz.yaml',
                                                yaml_key='search_tweets_api',
                                                env_overwrite=False)
    
    if include_retweets:
        
        search_term = search_term + ' lang:en'
    
    else:
        
        search_term = search_term + ' lang:en -is:retweet'
        
    
    
    rule = searchtweets.gen_rule_payload(search_term,
                                         results_per_call=500,
                                         from_date=start_date.strftime('%Y-%m-%d'),
                                         to_date=end_date.strftime('%Y-%m-%d'))     
        
    tweet_objects = []
    
    rs = searchtweets.ResultStream(rule_payload=rule,
                                   max_results=num_of_tweets,
                                   max_pages=math.ceil(num_of_tweets / 500.0),
                                   **search_args)
    
    start_time = time.time()
    
    # error handling & rate limiting for requests
    
    try:
        
        # generator 

        for i, tweet_object in enumerate(rs.stream(), start=1):
            
            # check every 500 requests, ie every page
        
            if i % 500 == 0:
            
                now = time.time()
                
                # four second delay
            
                if now < start_time + 4:
                
                    time.sleep(4 - (now - start_time))
                
                    start_time = time.time()
                
            tweet_objects.append(tweet_object)
    
    except Exception as e:
        
        print("Error from Twitter API: %s. Fetched %d tweet_objects" % (e, len(tweet_objects)))
    
    return tweet_objects

In [3]:
def tweet_obj_to_df(tweet_objects):
    
    '''
    takes a tweet object
    
    * appends text, author & other info the a dictionary
    
    * converts dictionary to dataframe
    
    returns dataframe with tweet info
    
    '''
    
    tweet_dict = {'text': [],
                  'author': []
                 }
    
    for tweet in tweet_objects:
        
        tweet_dict['text'].append(tweet.all_text)
        
        tweet_dict['author'].append(tweet.screen_name)
    
    tweets_df = pd.DataFrame(tweet_dict)
    
    return tweets_df